In [2]:
import cv2
import numpy as np
from keras.models import load_model
from keras.preprocessing.image import img_to_array

In [3]:
 #Load my model
custom_model = load_model("Model.keras")

emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

In [4]:
# Load the Haar Cascade Classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize the video capture
video_capture = cv2.VideoCapture(0)

In [ ]:
while True:
    ret, frame = video_capture.read()
    if not ret:
        print("Failed to retrieve frame. Exiting...")
        break

    labels = []

    # Convert grayscale frame to color (3 channels)
    frame_color = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame_color, (x, y), (x + w, y + h), (0, 255, 255), 2)
        roi_gray = gray[y:y + h, x:x + w]
        roi_gray = cv2.resize(roi_gray, (48, 48), interpolation=cv2.INTER_AREA)

        if np.sum([roi_gray]) != 0:
            # Convert grayscale ROI to color (3 channels)
            roi_color = cv2.cvtColor(roi_gray, cv2.COLOR_GRAY2BGR)

            roi = roi_color.astype('float') / 255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi, axis=0)

            prediction = custom_model.predict(roi)[0]  # Use the loaded model
            label = emotion_labels[prediction.argmax()]
            label_position = (x, y)
            cv2.putText(frame_color, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        else:
            cv2.putText(frame_color, 'No Faces', (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow('Emotion Detector', frame_color)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 53ms/step


1/1 [==============================] - 0s 81ms/step
